In [1]:
pip install duckdb

Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install jupysql
# pip install notebook

In [3]:
# If on windows, this only works if you're running Jupyter as an admin
pip install duckdb-engine

  Using cached duckdb_engine-0.13.1-py3-none-any.whl (47 kB)
  Using cached packaging-24.1-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 20.9
    Uninstalling packaging-20.9:
      Successfully uninstalled packaging-20.9
Note: you may need to restart the kernel to use updated packages.


In [82]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [107]:
import numpy as np
import pandas as pd
import seaborn as sns
import duckdb
import sqlalchemy
from datetime import date
import json
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
# Set up SQL connection
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [95]:
current_date = date.today().strftime('%Y-%m-%d')

In [8]:
# Replace with your filepath. Forward slashes work for Windows too
%sql duckdb:///C:/Users/Admin/billboard/raw_db.duckdb

In [14]:
%sql USE raw_db;
%sql USE raw_db.billboard;

,Success


In [15]:
%sql SELECT * FROM raw_db.billboard.artist LIMIT 10;

,artist_id,artist_name,pct_female,last_update
0,1,Zach Bryan,0.0,2024-08-23
1,2,Morgan Wallen,0.0,2024-08-23
2,3,Beyonce,1.0,2024-08-23
3,4,Megan Moroney,1.0,2024-08-23
4,5,Jelly Roll,0.0,2024-08-23
5,6,Luke Combs,0.0,2024-08-23
6,7,Dylan Scott,0.0,2024-08-23
7,8,Bailey Zimmerman,0.0,2024-08-23
8,9,Tucker Wetmore,0.0,2024-08-23
9,10,Cody Johnson,0.0,2024-08-23


In [23]:
# Update this for this week's rankings
rankings = [
    'A Bar Song (Tipsy)'
    , 'I Had Some Help'
    , 'Pour Me A Drink'
    , 'Cowgirls'
    , "Ain't No Love In Oklahoma"
    , 'Pink Skies'
    , 'Guy For That'
    , 'Lies Lies Lies'
    , 'Miles On It'
    , 'Austin'
    , 'Wrong Ones' # Post Malone feat Tim McGraw
    , 'Losers' # Post Malone feat Jelly Roll
    , 'I Am Not Okay'
    , 'California Sober' # Post Malone feat Chris Stapleton
    , '28'
    , "What Don't Belong To Me" # Post Malone
    , 'Finer Things' # Post Malone feat Hank Williams
    , 'Nosedive' # Post Malone feat Lainey Wilson
    , 'High Road'
    , 'Dirt Cheap'
    , 'Yours' # Post Malone
    , 'Have The Heart'
    , 'Bulletproof'
    , 'Whiskey Whiskey'
    , 'Goes Without Saying' # Post Malone feat Brad Paisley
    , 'You Look Like You Love Me'
    , "Missin' You Like This" # Post Malone feat Luke Combs
    , 'Hide My Gun' # Post Malone feat HARDY
    , "Devil I've Been" # Post Malone feat ERNEST
    , 'Chevrolet'
    , 'Lonely Road'
    , 'Hang Tight Honey'
    , 'Never Love You Again' # Post Malone feat Sierra Ferrell
    , "Wind Up Missin' You"
    , 'M-E-X-I-C-O' # Post Malone feat Billy Strings
    , 'Beautiful As You'
    , "Think I'm In Love With You"
    , 'Love You, Miss You, Mean It'
    , 'Right About You' # Post Malone
    , "She's Somebody's Daughter (Reimagined)"
    , 'American Nights'
    , 'Am I Okay?'
    , 'Holy Smokes'
    , 'Young Love & Saturday Nights'
    , 'Love You Again'
    , 'Sweet Dreams'
    , 'Devil You Know'
    , 'Worst Way'
    , 'Relapse'
    , 'Wanna Be Loved'
]

In [84]:
%%sql df_song <<
SELECT
    ss.song_id
    , ss.song_name
    , art.artist_name
    , s2a.relationship_type
    , ss.untracked_weeks_on_chart
FROM song ss
LEFT JOIN song_to_artist s2a
    ON ss.song_id = s2a.song_id
LEFT JOIN artist art
    ON s2a.artist_id = art.artist_id
;

In [86]:
df_song.head()

,song_id,song_name,artist_name,relationship_type,untracked_weeks_on_chart
0,1,I Remember Everything,Kacey Musgraves,Featured artist,21.0
1,9,Save Me,Lainey Wilson,Other main artist,34.0
2,17,Blackbiird,Reyna Roberts,Other main artist,0.0
3,21,Dirt Cheap,Cody Johnson,Lead artist,11.0
4,22,Think I'm In Love With You,Chris Stapleton,Lead artist,10.0


In [119]:
existing_song_to_rank = {}
new_song_to_rank = {}
counter = 0
for song, rank_ix in zip(rankings, range(1,50)):
    print(df_song.loc[df_song['song_name'] == song].to_markdown())
    song_id = input("Please enter the song_id or 'New Song'\n")
    if song_id.lower() == 'new song':
        new_song_to_rank[song] = rank_ix
    else:
        existing_song_to_rank[int(song_id)] = rank_ix
    
    # Still testing, no need to do the whole list
    counter += 1
    if counter > 12:
        break

|    |   song_id | song_name          | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:-------------------|:--------------|:--------------------|---------------------------:|
| 16 |        34 | A Bar Song (Tipsy) | Shaboozey     | Lead artist         |                          0 |
Please enter the song_id or 'New Song'
34
|    |   song_id | song_name       | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------|:--------------|:--------------------|---------------------------:|
| 20 |        38 | I Had Some Help | Morgan Wallen | Other main artist   |                          0 |
| 62 |        38 | I Had Some Help | Post Malone   | Lead artist         |                          0 |
Please enter the song_id or 'New Song'
38
|    |   song_id | song_name       | artist_name   | relationship_type   |   untracked_weeks_on_chart |
|---:|----------:|:----------------|:--------------|:--------------------|-

In [120]:
print(json.dumps(existing_song_to_rank, indent = 2))
print(json.dumps(new_song_to_rank, indent = 2))

{
  "34": 1,
  "38": 2,
  "52": 3,
  "32": 4,
  "40": 5,
  "43": 6,
  "64": 7,
  "58": 8,
  "37": 9,
  "27": 10,
  "48": 13
}
{
  "Wrong Ones": 11,
  "Losers": 12
}


In [121]:
## Add artists on any new songs that aren't in the artist table yet to its CSV

In [122]:
## Add all new songs to the song table CSV

In [123]:
## Add all new songs to the song_to_artist table CSV

In [124]:
## Convert new_song_to_rank to the id_to_rankix format of existing_song_to_rank

In [125]:
## Stack the two dicts of songs and insert to CSV with more info like current_date for last_update